In [34]:
!pip install geopandas
!pip install contextily

In [35]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [36]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [37]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [38]:
iso3 = "GUY"
file_path = f"/projects/my-public-bucket/AOIs/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [39]:
json_files = tileindex_df['s3path']
AOIs = json_files
len(AOIs)
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

'tile_num_21324'

In [40]:
AOIs[1]

'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_21325.geojson'

In [41]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result



Thanks, I'll run the GEDI L4A subsetter for tile_num_21324!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21325!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21502!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21503!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21504!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21505!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21506!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21508!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21682!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21683!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21684!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21685!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21686!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21687!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21688!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_21689!!
Thanks, I'll run the GED

In [42]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2A subsetter for tile_num_21324!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21325!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21502!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21503!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21504!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21505!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21506!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21508!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21682!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21683!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21684!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21685!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21686!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21687!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21688!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_21689!!
Thanks, I'll run the GED

In [43]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2B subsetter for tile_num_21324!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21325!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21502!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21503!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21504!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21505!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21506!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21508!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21682!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21683!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21684!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21685!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21686!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21687!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21688!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_21689!!
Thanks, I'll run the GED

In [9]:
## if necessary ## re-run failed subsets
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15929.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15749.geojson'
inputs = dict(
    aoi=each_aoi,
    doi="L4A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
    query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'ff05d990-1359-45ed-8ffa-e7c4980a191c'

In [21]:
##L2B
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17915.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18093.geojson'
inputs = dict(
    aoi=each_aoi,
    doi="L2B",
    lat="geolocation/lat_lowestmode",
    lon="geolocation/lon_lowestmode",
    beams="all",
    columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
    query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'4458c49d-c255-479d-b841-55376581ee9f'

In [16]:
## L4A

In [9]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


346


In [10]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/07/19"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_15932_L4A.gpkg...
Copying tile_num_15931_L4A.gpkg...
Copying tile_num_16110_L4A.gpkg...
Copying tile_num_16112_L4A.gpkg...
Copying tile_num_15930_L4A.gpkg...
Copying tile_num_16292_L4A.gpkg...
Copying tile_num_16291_L4A.gpkg...
Copying tile_num_17733_L4A.gpkg...
Copying tile_num_16111_L4A.gpkg...
Copying tile_num_18092_L4A.gpkg...
Copying tile_num_17915_L4A.gpkg...
Copying tile_num_18272_L4A.gpkg...
Copying tile_num_18094_L4A.gpkg...
Copying tile_num_17914_L4A.gpkg...
Copying tile_num_17911_L4A.gpkg...
Copying tile_num_17913_L4A.gpkg...
Copying tile_num_18096_L4A.gpkg...
Copying tile_num_18091_L4A.gpkg...
Copying tile_num_17912_L4A.gpkg...
Copying tile_num_18093_L4A.gpkg...
Copying tile_num_18095_L4A.gpkg...
Copying tile_num_18089_L4A.gpkg...
Copying tile_num_17910_L4A.gpkg...
Copying tile_num_18812_L4A.gpkg...
Copying tile_num_18813_L4A.gpkg...
Copying tile_num_18090_L4A.gpkg...
Copying tile_num_18273_L4A.gpkg...
Copying tile_num_18270_L4A.gpkg...
Copying tile_num_182

In [20]:
## L2A

In [11]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


39


In [12]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/07/19"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_15932_L2A.gpkg...
Copying tile_num_16112_L2A.gpkg...
Copying tile_num_15749_L2A.gpkg...
Copying tile_num_18452_L2A.gpkg...
Copying tile_num_15931_L2A.gpkg...
Copying tile_num_15929_L2A.gpkg...
Copying tile_num_16111_L2A.gpkg...
Copying tile_num_18273_L2A.gpkg...
Copying tile_num_18272_L2A.gpkg...
Copying tile_num_16292_L2A.gpkg...
Copying tile_num_16291_L2A.gpkg...
Copying tile_num_18092_L2A.gpkg...
Copying tile_num_18274_L2A.gpkg...
Copying tile_num_18454_L2A.gpkg...
Copying tile_num_17914_L2A.gpkg...
Copying tile_num_18096_L2A.gpkg...
Copying tile_num_18093_L2A.gpkg...
Copying tile_num_18453_L2A.gpkg...
Copying tile_num_17733_L2A.gpkg...
Copying tile_num_17915_L2A.gpkg...
Copying tile_num_18094_L2A.gpkg...
Copying tile_num_17912_L2A.gpkg...
Copying tile_num_18095_L2A.gpkg...
Copying tile_num_17913_L2A.gpkg...
Copying tile_num_15930_L2A.gpkg...
Copying tile_num_18271_L2A.gpkg...
Copying tile_num_18451_L2A.gpkg...
Copying tile_num_18275_L2A.gpkg...
Copying tile_num_161

In [23]:
## L2B

In [26]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


143


In [28]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/07/19"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2B.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

tile_num_15749_L2B.gpkg already exists in the target directory. Skipping...
tile_num_15929_L2B.gpkg already exists in the target directory. Skipping...
tile_num_15932_L2B.gpkg already exists in the target directory. Skipping...
tile_num_15931_L2B.gpkg already exists in the target directory. Skipping...
tile_num_15930_L2B.gpkg already exists in the target directory. Skipping...
tile_num_16112_L2B.gpkg already exists in the target directory. Skipping...
tile_num_16292_L2B.gpkg already exists in the target directory. Skipping...
tile_num_18812_L2B.gpkg already exists in the target directory. Skipping...
tile_num_16291_L2B.gpkg already exists in the target directory. Skipping...
tile_num_18273_L2B.gpkg already exists in the target directory. Skipping...
tile_num_18272_L2B.gpkg already exists in the target directory. Skipping...
tile_num_18092_L2B.gpkg already exists in the target directory. Skipping...
tile_num_17915_L2B.gpkg already exists in the target directory. Skipping...
tile_num_180